In [1]:
# import libraries

import pandas as pd
import tweepy
import os
from dotenv import load_dotenv
from requests_oauthlib import OAuth1Session
import requests
import json
from datetime import datetime, timedelta
import tweetnlp
import openai

c:\Users\Erjon\Documents\1. GERMANY\1. HTW Berlin\3. Courses\3.1 - Project Management and Data Analytics Lab\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-12 01:26:28,402	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-11-12 01:26:28,653	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


### **Twitter Api Connection**

##### **Posting a tweet using the free tier**

In [2]:
# .env did not work for some reason, so I created a keys.py file with the keys

from keys import *
client = tweepy.Client( bearer_token=bearer_token, 
                        consumer_key=consumer_key, 
                        consumer_secret=consumer_secret, 
                        access_token=access_token, 
                        access_token_secret=access_token_secret,
                        )

In [113]:
'''
DONT RUN THIS CELL UNLESS YOU WANT TO POST A TWEET

The twitter API v1.1 is being deprecated, so I am using the v2 API. 

The documentation for the v2 is very poor.
Since V1.1 has been around forever, most of the examples online are for the v1.1.
Same is about LLM like chatgpt or bard. They have been trained mostly on v1.1 data so they are useless for v2.

The following example was taken from the Twitter API documentation found in the following link:
https://github.com/twitterdev/Twitter-API-v2-sample-code

'''


'''
# Be sure to add replace the text of the with the text you wish to Tweet. 
payload = {"text": "First tweet using Twitter API v2 and Python"}

# Get request token
request_token_url = "https://api.twitter.com/oauth/request_token?oauth_callback=oob&x_auth_access_type=write"
oauth = OAuth1Session(consumer_key, client_secret=consumer_secret)

try:
    fetch_response = oauth.fetch_request_token(request_token_url)
except ValueError:
    print(
        "There may have been an issue with the consumer_key or consumer_secret you entered."
    )

resource_owner_key = fetch_response.get("oauth_token")
resource_owner_secret = fetch_response.get("oauth_token_secret")
print("Got OAuth token: %s" % resource_owner_key)

# Get authorization
base_authorization_url = "https://api.twitter.com/oauth/authorize"
authorization_url = oauth.authorization_url(base_authorization_url)
print("Please go here and authorize: %s" % authorization_url)
verifier = input("Paste the PIN here: ")

# Get the access token
access_token_url = "https://api.twitter.com/oauth/access_token"
oauth = OAuth1Session(
    consumer_key,
    client_secret=consumer_secret,
    resource_owner_key=resource_owner_key,
    resource_owner_secret=resource_owner_secret,
    verifier=verifier,
)
oauth_tokens = oauth.fetch_access_token(access_token_url)

access_token = oauth_tokens["oauth_token"]
access_token_secret = oauth_tokens["oauth_token_secret"]

# Make the request
oauth = OAuth1Session(
    consumer_key,
    client_secret=consumer_secret,
    resource_owner_key=access_token,
    resource_owner_secret=access_token_secret,
)

# Making the request
response = oauth.post(
    "https://api.twitter.com/2/tweets",
    json=payload,
)

if response.status_code != 201:
    raise Exception(
        "Request returned an error: {} {}".format(response.status_code, response.text)
    )

print("Response code: {}".format(response.status_code))

# Saving the response as JSON
json_response = response.json()
print(json.dumps(json_response, indent=4, sort_keys=True))

'''

'\n# Be sure to add replace the text of the with the text you wish to Tweet. You can also add parameters to post polls, quote Tweets, Tweet with reply settings, and Tweet to Super Followers in addition to other features.\npayload = {"text": "First tweet using Twitter API v2 and Python"}\n\n# Get request token\nrequest_token_url = "https://api.twitter.com/oauth/request_token?oauth_callback=oob&x_auth_access_type=write"\noauth = OAuth1Session(consumer_key, client_secret=consumer_secret)\n\ntry:\n    fetch_response = oauth.fetch_request_token(request_token_url)\nexcept ValueError:\n    print(\n        "There may have been an issue with the consumer_key or consumer_secret you entered."\n    )\n\nresource_owner_key = fetch_response.get("oauth_token")\nresource_owner_secret = fetch_response.get("oauth_token_secret")\nprint("Got OAuth token: %s" % resource_owner_key)\n\n# Get authorization\nbase_authorization_url = "https://api.twitter.com/oauth/authorize"\nauthorization_url = oauth.authoriza

##### **Searching and pulling tweets (pseudocode)**

In [33]:
search_url = "https://api.twitter.com/2/tweets/search/recent"

excluded_authors = ["tlrailuk", "anotherUser"]
excluded_authors_query = " ".join(f"-from:{author}" for author in excluded_authors)

# Calculate the end of the previous day
end_time = datetime.utcnow().replace(hour=0, minute=0, second=0, microsecond=0) - timedelta(seconds=1)
# Calculate the start time as 7 days before the end time
start_time = end_time - timedelta(days=7)

# Format dates in ISO 8601 format
start_time_str = start_time.strftime("%Y-%m-%dT%H:%M:%SZ")
end_time_str = end_time.strftime("%Y-%m-%dT%H:%M:%SZ")

# Parameters: exclude authors, no retweets
# Returns: author_id, created_at, text, username, from the last 7 days

query_params = {
    'query': f'@tlrailuk {excluded_authors_query} -is:retweet', 
    'tweet.fields': 'author_id,created_at,text',
    'user.fields': 'username', # only temporary. Will be removed once we are done with tests
    'start_time': start_time_str,
    'end_time': end_time_str
}

def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

def connect_to_endpoint(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


def main():
    '''(remove the triple quotes to run the code)
    json_response = connect_to_endpoint(search_url, query_params) # This is the response from the API
    print(json.dumps(json_response, indent=4, sort_keys=True))
    '''

    # this reads and turns into a dataframe a mock response from the API

    # Load the JSON file
    with open("../../Data/Raw/mock_tweets.json", "r") as file:
        data = json.load(file)

    # Create a DataFrame from the nested 'data' key
    df = pd.json_normalize(data, record_path=['data'])
    return df
  
  
if __name__ == "__main__":
    df_response = main()


In [4]:
df_response.head(3)

,text,created_at,author_id,username
0,Delayed again! @tlrailuk needs to sort out the...,2023-03-01T08:30:00Z,1001,user1
1,Unbelievable! The doors on @tlrailuk train did...,2023-03-01T09:00:00Z,1002,user2
2,Why are the toilets always out of order on @tl...,2023-03-01T09:30:00Z,1003,user3


### **Sentiment Analysis**

In [34]:
model = tweetnlp.load_model('sentiment')

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [47]:
# use tweetnlp model to predict the sentiment of the tweets
# keep only the negative and neutral tweets

df_response['sentiment'] = df_response['text'].apply(lambda x: model.predict(x)['label'])
df_response_complaints = df_response[(df_response['sentiment'] == 'negative') | (df_response['sentiment'] == 'neutral')]
df_response_complaints.head(3)

,text,created_at,author_id,username,sentiment
0,Delayed again! @tlrailuk needs to sort out the...,2023-03-01T08:30:00Z,1001,user1,negative
1,Unbelievable! The doors on @tlrailuk train did...,2023-03-01T09:00:00Z,1002,user2,negative
2,Why are the toilets always out of order on @tl...,2023-03-01T09:30:00Z,1003,user3,negative


### **Filter Out Delays**

In [46]:
# filter out the tweets that have the word 'delay' in them

df_response_filter = df_response_complaints[~df_response_complaints['text'].str.contains('delay', case=False, na=False)]
df_response_filter.head(3)

,text,created_at,author_id,username,sentiment
1,Unbelievable! The doors on @tlrailuk train did...,2023-03-01T09:00:00Z,1002,user2,negative
2,Why are the toilets always out of order on @tl...,2023-03-01T09:30:00Z,1003,user3,negative
3,Completely unprofessional staff at @tlrailuk t...,2023-03-01T10:00:00Z,1004,user4,negative


In [45]:
df_response_analysis = df_response_filter.copy()
df_response_analysis['date'] = df_response_analysis['created_at'].apply(lambda x: x.split('T')[0])
df_response_analysis['time'] = df_response_analysis['created_at'].apply(lambda x: x.split('T')[1].rstrip('Z'))
df_response_analysis.drop(['created_at'], axis=1, inplace=True)
df_response_analysis.head(3)

,text,author_id,username,sentiment,date,time
1,Unbelievable! The doors on @tlrailuk train did...,1002,user2,negative,2023-03-01,09:00:00
2,Why are the toilets always out of order on @tl...,1003,user3,negative,2023-03-01,09:30:00
3,Completely unprofessional staff at @tlrailuk t...,1004,user4,negative,2023-03-01,10:00:00


### **LLM Analysis**

In [38]:
OPENAI_API_KEY = OPENAI_API_KEY

In [44]:
# Constructing the prompt for OpenAI
prompt = """
As a senior train maintenance official, I am consolidating passenger complaints for efficient resolution. 
Below are various complaints from train passengers, each with a specific topic. 
I need a structured summary for each unique topic, combining similar complaints and providing actionable suggestions for resolution starting from the next day. 
Please group the complaints by their topics, summarize the key points, and suggest collective resolutions.
End the reponse with the following line: "Please note that these suggestions are based on the provided information and may need further assessment and customization. Help us improve our service by providing feedback on your experience."



Complaints:
"""

complaint_summary = df_response_analysis.groupby(['text', 'date']).size().reset_index(name='count')

for index, row in complaint_summary.iterrows():
    prompt += f"- Date Created: {row['date']} - Complaint: {row['text']} (Count: {row['count']})\n"

prompt += """
For each complaint, provide:
1. Summary of the complaint in a very concise manner
2. Topic (1 or 2 words)
3. Suggestion for resolution with actionable steps starting from the next day

Please structure your response as follows:
- Date Created:
- Topic:
- Count of complaints for this topic:
- Summary:
- Suggestion:
"""

# Set up OpenAI API key
openai.api_key = OPENAI_API_KEY

# Generate response from OpenAI
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",  # Replace with the appropriate chat model
    messages=[
        {"role": "system", "content": "You are a senior train maintenance official."},
        {"role": "user", "content": prompt}
    ]
)

# Print the response
print(response.choices[0].message["content"])
data = response.choices[0].message["content"]

- Date Created: 2023-03-01
- Topic: Overcrowding
- Count of complaints for this topic: 1
- Summary: Passengers reported an overcrowded train experience.
- Suggestion: Increase the number of train services during peak hours to alleviate overcrowding. Consider adding additional carriages or trains to high-demand routes. Conduct a thorough analysis to determine which routes experience the highest levels of overcrowding and prioritize improvements accordingly.

- Date Created: 2023-03-01
- Topic: Staff professionalism
- Count of complaints for this topic: 1
- Summary: Passengers expressed their disappointment with unprofessional staff behavior.
- Suggestion: Conduct refresher training programs for staff members, focusing on customer service skills, professionalism, and effective communication. Emphasize the importance of maintaining a positive and helpful attitude towards passengers. Regularly monitor and evaluate staff performance to ensure consistent professionalism.

- Date Created: 202

In [50]:
# Split the data into separate entries
entries = data.strip().split("- Date Created:")

# Initialize lists to hold the data
dates = []
topics = []
counts = []
summaries = []
suggestions = []

# Process each entry
for entry in entries:
    if entry:  # skip empty strings
        lines = entry.strip().split("\n- ")
        date, topic, count, summary, suggestion = lines
        dates.append(date.replace("Date Created: ", "").strip())
        topics.append(topic.replace("Topic: ", "").strip())
        counts.append(int(count.split(": ")[1]))
        summaries.append(summary.replace("Summary: ", "").strip())
        suggestions.append(suggestion.replace("Suggestion: ", "").strip())

# Create a DataFrame
df = pd.DataFrame({
    'Date': dates,
    'Topic': topics,
    'Count': counts,
    'Summary': summaries,
    'Suggestion': suggestions
})

df.head()  # To display the first few rows of the DataFrame

,Date,Topic,Count,Summary,Suggestion
0,2023-03-01,Overcrowding,1,Passengers reported an overcrowded train exper...,Increase the number of train services during p...
1,2023-03-01,Staff professionalism,1,Passengers expressed their disappointment with...,Conduct refresher training programs for staff ...
2,2023-03-01,Ticket payment issue,1,A passenger faced double charges for their tic...,Implement a thorough review of the ticketing s...
3,2023-03-01,Non-functioning toilets,2,Passengers reported encountering non-functioni...,Establish a robust maintenance schedule for tr...
4,2023-03-01,Door malfunctions,1,A passenger experienced a train where the door...,Implement recurrent door maintenance checks to...
